In [ ]:
## INITIALIZE ##
from dotenv import load_dotenv
import os
from langgraph_app import app_graph
from IPython.display import Markdown, display

# Load environment variables from .env file in the current directory
load_dotenv()

# Enable automatic reloading
%load_ext autoreload
%autoreload 2

In [ ]:
## OCR ##
# Example: run the LangGraph workflow
images = ["images/page1.jpg", "images/page2.jpg", "images/page3.jpg", "images/page4.jpg"]  # Replace with your actual image file paths
markdown = app_graph.run_node("OCR", images)
display(Markdown(markdown))

In [ ]:
## EXTRACT DIAGNOSES ##
diagnoses = app_graph.run_node("ExtractDiagnoses", model="gpt-4o-mini")

In [ ]:
## EXTRACT MEDICATIONS ##
medications = app_graph.run_node("ExtractMedications", model="gpt-4o-mini")


In [ ]:
## FIX MEDICATIONS ##
fixed_medications = app_graph.run_node("FixMedications", model="gpt-4o")


In [ ]:
html_summary = app_graph.run_node("AddSummaryPills")
print(html_summary)